In [ ]:
import arviz as az
import os

os.environ["JAX_ENABLE_X64"] = "1"

import jax
import jax.numpy as jnp
import polars as pl
import numpy as np
import bambi as bmb

In [6]:
os.getcwd()

'/Users/techen/biosustain/course/bayesian_statistics/projects/bayesian_statistics_for_computational_biology/project'

In [16]:
data = pl.read_csv("./data/6-Pokemon_gen2.csv")
data.head()

,Name,Primary Typing,Secondary Typing,Health,Attack,Defense,Special Attack,Special Defense,Speed
i64,str,str,str,i64,i64,i64,i64,i64,i64
206,"""chikorita""","""grass""",null,45,49,65,49,65,45
207,"""bayleef""","""grass""",null,60,62,80,63,80,60
208,"""meganium""","""grass""",null,80,82,100,83,100,80
209,"""cyndaquil""","""fire""",null,39,52,43,60,50,65
210,"""quilava""","""fire""",null,58,64,58,80,65,80


In [17]:
data.select(
    [
        pl.col("Name"),
        pl.col("Health"),
        pl.col("Attack"),]
)

Name,Health,Attack
str,i64,i64
"""chikorita""",45,49
"""bayleef""",60,62
"""meganium""",80,82
"""cyndaquil""",39,52
"""quilava""",58,64
…,…,…
"""tyranitar""",100,134
"""tyranitar-mega""",100,164
"""lugia""",106,90


In [27]:
def clr_transform(row: dict) -> list[float]:
    values = list(row.values())
    log_stats = np.log(values)
    geometric_mean = np.mean(log_stats)
    return list(log_stats - geometric_mean)

old_col = [
    'Health', 'Attack', 'Defense',
    'Special Attack', 'Special Defense', 'Speed'
]

new_col = [
        'clr_Health', 'clr_Attack', 'clr_Defense',
        'clr_Special_Attack', 'clr_Special_Defense', 'clr_Speed'
    ]

clr_data = data.with_columns(
    pl.struct(old_col)
    .map_elements(clr_transform)
    .alias('clr_stats')
)

clr_data = clr_data.with_columns(
    pl.col('clr_stats').list.to_struct(fields=new_col)
).unnest('clr_stats')

/var/folders/rm/0tp2_wjs4hl3t3t2bgrztxbr0000gp/T/ipykernel_43790/2422304245.py:17: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  clr_data = data.with_columns(


In [ ]:
clr_data

,Name,Primary Typing,Secondary Typing,Health,Attack,Defense,Special Attack,Special Defense,Speed,clr_Health,clr_Attack,clr_Defense,clr_Special_Attack,clr_Special_Defense,clr_Speed
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
206,"""chikorita""","""grass""",null,45,49,65,49,65,45,-0.150961,-0.065803,0.216764,-0.065803,0.216764,-0.150961
207,"""bayleef""","""grass""",null,60,62,80,63,80,60,-0.109491,-0.076701,0.178191,-0.060701,0.178191,-0.109491
208,"""meganium""","""grass""",null,80,82,100,83,100,80,-0.084632,-0.05994,0.138511,-0.047818,0.138511,-0.084632
209,"""cyndaquil""","""fire""",null,39,52,43,60,50,65,-0.262565,0.025117,-0.164927,0.168218,-0.014104,0.248261
210,"""quilava""","""fire""",null,58,64,58,80,65,80,-0.142592,-0.044152,-0.142592,0.178992,-0.028648,0.178992
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
313,"""tyranitar""","""rock""","""dark""",100,134,110,95,100,61,0.026268,0.318938,0.121578,-0.025025,0.026268,-0.468028
314,"""tyranitar-mega""","""rock""","""dark""",100,164,150,95,120,71,-0.114783,0.379913,0.290682,-0.166077,0.067538,-0.457274
315,"""lugia""","""psychic""","""flying""",106,90,130,90,154,110,-0.047899,-0.211528,0.156197,-0.211528,0.325615,-0.010857


In [ ]:
clr_data.unpivot(index=["Name"])

Name,variable,value
str,str,str
"""chikorita""","""""","""206"""
"""bayleef""","""""","""207"""
"""meganium""","""""","""208"""
"""cyndaquil""","""""","""209"""
"""quilava""","""""","""210"""
…,…,…
"""tyranitar""","""clr_Speed""","""-0.46802801807558225"""
"""tyranitar-mega""","""clr_Speed""","""-0.45727352618075834"""
"""lugia""","""clr_Speed""","""-0.010857276446620823"""
